In [ ]:
import pandas as pd
import os
import numpy as np

# Data Gathering

In [ ]:
os.chdir("../Raw Data")

In [ ]:
df = pd.concat([pd.read_csv("202301-divvy-tripdata.csv"), pd.read_csv("202302-divvy-tripdata.csv"), pd.read_csv("202303-divvy-tripdata.csv"),
               pd.read_csv("202304-divvy-tripdata.csv"), pd.read_csv("202305-divvy-tripdata.csv"), pd.read_csv("202306-divvy-tripdata.csv"),
               pd.read_csv("202307-divvy-tripdata.csv"), pd.read_csv("202308-divvy-tripdata.csv"), pd.read_csv("202309-divvy-tripdata.csv"),
               pd.read_csv("202310-divvy-tripdata.csv"), pd.read_csv("202311-divvy-tripdata.csv"), pd.read_csv("202312-divvy-tripdata.csv")])

In [ ]:
df.to_csv("2023-divvy-tripdata.csv")

In [ ]:
df = pd.read_csv("2023-divvy-tripdata.csv")

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.drop("Unnamed: 0", axis = 1, inplace =True)

In [ ]:
df[df['start_station_name'].isna()]

# Data Processing

## Cleaning 
- duplicate (ride_id)
- null/missing data (except start_station_id, and end_station_id)
## Validation
- inconsistency (ended_at < started_at)
- check latitude and longitude <br>
  $\quad$ Relevent link: Chicago Maphttps://wikimap.toolforge.org/?lang=en&page=Chicago<br>
  $\quad$ Chicago coordinate (Approximate):<br>
    $\quad$$\quad$lat_max: 42.02296 <br>
    $\quad$$\quad$lat_min: 41.64378 <br>
    $\quad$$\quad$lng_max: -87.52509 <br>
    $\quad$$\quad$lng_min: -87.94014 <br>
        
## Add Attributes
- weekday (Mon-Sun)
- Month (Jan-Dec)
- holiday (e.g.Christimas, Thanksgiving)   <br>
    relevent link: https://www.independent.co.uk/life-style/federal-holidays-2023-dates-december-b2468299.html
- duration in minute
- distance in km
- speed

In [ ]:
df[df.duplicated()]
#df[df.duplicated(subset = 'ride_id')]

In [ ]:
df.isna().sum()

In [ ]:
#929202/5719877
6990/5719877

- Based on above process, there is no duplicate in the dataset.
- About 16% station name and id are missing. This may be caused by user did not start/stop at the designated stations. It could happen when there is no station nearby or the bike is stole.
- 6990 pairs of end coordinates are missing. Based the distribution of end latitude and longtitude, it is in a small range with no outliers. Also, cconsidering it only takes 1.22% dataset, it can be replace with the mean of end latitude and longtitude.
- Station ids and ride id can be removed, because they won't contribute to our analysis.
- Station name should be saved to do further analysis, rename them based on the coordinate.(lat_lng)

In [ ]:
#Calculate the means of the columns having missing values
end_lat_mean, end_lng_mean = (df['end_lat'].mean(), df['end_lng'].mean()) 

# Replace NaNs in column end_lat and end_lng with means values
df['end_lat'].fillna(value = end_lat_mean, inplace = True)
df['end_lng'].fillna(value = end_lng_mean, inplace = True)

In [ ]:
df.drop(["ride_id","end_station_id", "start_station_id"], axis = 1, inplace = True)

######################### Station Name (Unsaved Change) ################################

In [ ]:
df1 = df[df['start_station_name'].isna() | df['end_station_name'].isna()]
#df1['end_station_name'].fillna(value = df1['end_lat'].astype(str) + df1['end_lng'].astype(str), inplace = True)
df1['start_station_name'] = df1['start_station_name'].where(df1['start_station_name'].isna() == False, df1['start_lat'].astype(str)+df['start_lng'].astype(str))
df1['end_station_name'] = df1['end_station_name'].where(df1['end_station_name'].isna() == False, df1['end_lat'].astype(str)+df['end_lng'].astype(str))

## Validation
- inconsistency (ended_at < started_at)
- check latitude and longitude <br>
  $\quad$ Relevent link: Chicago Map https://wikimap.toolforge.org/?lang=en&page=Chicago<br>
  $\quad$ Chicago coordinate (Approximate):<br>
    $\quad$$\quad$lat_max: 42.02296 <br>
    $\quad$$\quad$lat_min: 41.64378 <br>
    $\quad$$\quad$lng_max: -87.52509 <br>
    $\quad$$\quad$lng_min: -87.94014 <br>

In order to validate the start and end time, converting data type from object to Datetime is required.<br>
Considering this analysis only focus on Chicago area, only records of trips that occurred in Chicago should be selected. It will be achieved by using extreme coordinate of Chicago.

In [ ]:
# Convert timestamps to Datetime format
df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Filter out invalide rides
df = df[df['ended_at'] > df['started_at']] #272

# Filter out rides with out-of-Chicago coordinate
df = df[(df['start_lat'] < 42.064955 ) & (df['start_lat'] > 41.64378)] #226
df = df[(df['end_lat'] < 42.064955 ) & (df['end_lat'] > 41.64378)] #332
df = df[(df['start_lng'] < -87.52) & (df['start_lng'] > -87.94014)] #1
df = df[(df['end_lng'] < -87.52) & (df['end_lng'] > -87.94014)] #15

In [ ]:
df[(df['end_lng']>-87.52) | (df['end_lng']<-87.94014)] #OH Charging Stx - Test

In [ ]:
df[(df['end_station_name'] == 'OH Charging Stx - Test') | (df['start_station_name'] == 'OH Charging Stx - Test')]

In [ ]:
df.iloc[3413675, 7] = 41.780506
df.iloc[3413675, 8] = -87.586853

df.iloc[2149865, 7] = 41.86257
df.iloc[2149865, 8] = -87.679935

df.iloc[2159176, 7] = 41.86257
df.iloc[2159176, 8] = -87.679935

During checking data quality, we removed total **1826** lines of invalid data, including 272 invalid rides (*end time is ahead of start time*), and 1554 rides that are out of Chicago.<br>
While filtering ride out of the range, there are 3 records contains station name, but no coordinates. This may be caused by unstable Internet or other technical issue. By searching station name in the data set, it has been manually updated.

## Add Attributes
- duration in minute
- distance in km
- speed
- weekday (Mon-Sun)
- Month (Jan-Dec)
- (Optional) holiday (e.g.Christimas, Thanksgiving)   <br>
    relevent link: https://www.independent.co.uk/life-style/federal-holidays-2023-dates-december-b2468299.html


In [ ]:
# Calculate ride durations
df['duration'] = df['ended_at'] - df['started_at']

# Convert ride durations into minutes
df['duration_m'] = df['duration'].dt.total_seconds() / 60

# Convert ride durations into hours
df['duration_h'] = df['duration'].dt.total_seconds() / 3600

In [ ]:
conda install -c conda-forge pyproj

In [ ]:
# Calculate distance in km
from pyproj import Geod

def get_distance(start_lat, start_lng, end_lat, end_lng):
    g = Geod(ellps='WGS84')
    # 2D distance in meters with longitude, latitude of the points
    azimuth1, azimuth2, distance_2d = g.inv(start_lat, start_lng, end_lat, end_lng)
    return distance_2d

In [ ]:
df['distance'] = get_distance(df['start_lat'].tolist(), df['start_lng'].tolist(), df['end_lat'].tolist(), df['end_lng'].tolist())

In [ ]:
df['distance'] = df['distance'] / 1000

In [ ]:
# Extract day of the week for each ride from the 'started_at_datetime' value (Monday = 0, Tuesday = 1, etc.)
df['day_of_week_num'] = df['started_at'].dt.dayofweek

# Extract date (start) for each ride
df['day_of_month'] = df['started_at'].dt.day

# Extract hour (start) for each ride
df['start_hour'] = df['started_at'].dt.hour

# Map week days values
week_days = {0:"Monday", 1:"Tuesday", 2:"Wednesday", 3:"Thursday", 4:"Friday", 5:"Saturday", 6:"Sunday"}
df['day_of_week'] = df['day_of_week_num'].map(week_days)

In [ ]:
# Calculate speed
df['ride_speed'] = df['distance'] / df['duration_h']

In [ ]:
df

In [ ]:
# Reset index and print dataframe preview
df.reset_index(drop=True, inplace = True)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df[(df['distance'] == 0)]

In [ ]:
#df[(df['duration_m'] < 1)  & (df['distance'] == 0)]#81954
df[(df['duration_m'] < 1)  & (df['distance'] == 0) & (df['start_station_name'] == df['end_station_name'])] #41142

By observing the statistics, the minimum of ride speed is 0. Considering speed is determined by distance and duration in hour. It is likely caused by distance is equal to 0. This hypothesis is proven by filtering trips with same output where distance is 0 or speed is 0. (287768 rows)<br>
When we pull out the data, there are over 280k trips. Based on that, it is unlikely to be technical problems but normal case.<br>

- Senario 1 (Duration > 1 minute & start place = end place):</br>
    The trip has same departure and arrival location. Because of the privacy terms, there is no data about stops and detailed trip data. In senario 1, even though the speed may not accurately reflect the real story, those data are still valid.
- Senario 2 (Duration < 1 minute & start place = end place):</br>
    The trip has same departure and arrival location, and the duration is less than 1 minute. It is likely to be accidental check-ins or check-outs. In senario 2, those data will not contribute to the analysis; thus, removing them to avoid the distortion caused by it.
- Senario 3 (108548 rows: Duration > 1 minute & start place != end place & distance = 0):<br>
    The departure and arrival names are different, and the latitudes and longitudes are same. It is likely cause by technical issue. <br>
- Senario 4 (Duration < 1 minute & start place != end place & distance = 0) <br>
    * Special case: Through further investigation, there are several pairs of similar station name inputs.(e.g. start_station_name: Wilton Ave & Diversey Pkwy* & end_station_name: Wilton Ave & Diversey Pkwy; start_station_name: Wilton Ave & Diversey Pkwy (Temp) & end_station_name: Wilton Ave & Diversey Pkwy) Because how the station name is define is unclear, let's assume different station name refers to different place, even if they are similar.<br>


To solve this inconsistency, remove rides less then 1 minute would be reasonable to remve unrealistic trips.

In [ ]:
# for x in df["end_station_name"].unique():
#     if (type(x) is str):
#         if 'Temp' in x:
#             print(x)

In [ ]:
# Filter out rides less than 1 minute or over 24 hours
df = df[(df['duration_m'] > 1) & (df['duration_m'] < 24)]

Based on the regulation published in 2017, the speed limit for bicycle is 30 mph(approx.  48 km/h).<br>
Revelent link: https://www.chicago.gov/content/dam/city/depts/bacp/publicvehicleinfo/publicchauffer/TipsforMotorist03072017.pdf

In [ ]:
# Filter out speed greater than 48 km/h
df = df[df['ride_speed'] <= 48]

In [ ]:
# Reset index and print dataframe preview
df.reset_index(drop = True, inplace = True)

In [ ]:
# Save the processed data
df.to_csv("2023-divvy-tripdata_processed.csv")